In [1]:
# https://www.kaggle.com/datasets/prapannadhungel/nepali-currency-dataset

In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

2025-03-19 18:54:59.222245: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1742390699.246916   78234 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1742390699.254753   78234 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-19 18:54:59.280648: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    rotation_range=20,  # Rotate images slightly
    width_shift_range=0.2,  # Shift images horizontally
    height_shift_range=0.2,  # Shift images vertically
    shear_range=0.2,  # Shear effect
    zoom_range=0.2,  # Zoom into images
    horizontal_flip=True  # Flip images horizontally
)


In [5]:
train_generator = train_datagen.flow_from_directory(
    'Dataset/train',  # Path to dataset
    target_size=(224, 224),  # Ensure input matches model shape
    batch_size=32,
    class_mode='binary',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    'Dataset/test',  # Path to dataset
    target_size=(224, 224),  # Ensure input matches model shape
    batch_size=32,
    class_mode='binary',
    subset='validation'
)


Found 199 images belonging to 2 classes.
Found 20 images belonging to 2 classes.


In [6]:
train_generator.class_indices

{'fake': 0, 'real': 1}

In [7]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Step 4: Build the model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D(2, 2),
    
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(4, activation='softmax')  # 4 classes
])

# Compile the model for multi-class classification
model.compile(
    loss='sparse_categorical_crossentropy',  # Change from binary to categorical loss
    optimizer='adam',
    metrics=['accuracy']
)


train_steps_per_epoch = int(np.ceil(train_generator.samples / train_generator.batch_size))
validation_steps_per_epoch = int(np.ceil(validation_generator.samples / validation_generator.batch_size))

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_steps_per_epoch,
    epochs=20,
    validation_data=validation_generator,
    validation_steps=validation_steps_per_epoch,
    verbose=1
)


/home/jonah/.local/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
I0000 00:00:1742390703.645918   78234 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 2784 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1650, pci bus id: 0000:01:00.0, compute capability: 7.5
/home/jonah/.local/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20


I0000 00:00:1742390708.121277   78312 service.cc:148] XLA service 0x7bc3200062e0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1742390708.121344   78312 service.cc:156]   StreamExecutor device (0): NVIDIA GeForce GTX 1650, Compute Capability 7.5
2025-03-19 18:55:08.188135: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1742390708.416048   78312 cuda_dnn.cc:529] Loaded cuDNN version 90300
2025-03-19 18:55:08.925089: I external/local_xla/xla/service/gpu/autotuning/conv_algorithm_picker.cc:557] Omitted potentially buggy algorithm eng14{k25=0} for conv (f32[7,32,222,222]{3,2,1,0}, u8[0]{0}) custom-call(f32[7,3,224,224]{3,2,1,0}, f32[32,3,3,3]{3,2,1,0}, f32[32]{0}), window={size=3x3}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBiasActivationForward", backend_config={"cudnn_conv_backend_config":{"activ

1/7 ━━━━━━━━━━━━━━━━━━━━ 37s 6s/step - accuracy: 0.0000e+00 - loss: 1.4483

I0000 00:00:1742390712.382821   78312 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
2025-03-19 18:55:13.140474: I external/local_xla/xla/service/gpu/autotuning/conv_algorithm_picker.cc:557] Omitted potentially buggy algorithm eng14{k25=0} for conv (f32[32,32,222,222]{3,2,1,0}, u8[0]{0}) custom-call(f32[32,3,224,224]{3,2,1,0}, f32[32,3,3,3]{3,2,1,0}, f32[32]{0}), window={size=3x3}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBiasActivationForward", backend_config={"cudnn_conv_backend_config":{"activation_mode":"kNone","conv_result_scale":1,"leakyrelu_alpha":0,"side_input_scale":0},"force_earliest_schedule":false,"operation_queue_id":"0","wait_on_operation_queues":[]}
2025-03-19 18:55:13.261363: I external/local_xla/xla/service/gpu/autotuning/conv_algorithm_picker.cc:557] Omitted potentially buggy algorithm eng14{k25=0} for conv (f32[32,64,109,109]{3,2,1,0}, u8[0]{0}) custom-call(f32[32,32,111,111]{3,

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.3911 - loss: 3.3332

2025-03-19 18:55:23.901536: I external/local_xla/xla/service/gpu/autotuning/conv_algorithm_picker.cc:557] Omitted potentially buggy algorithm eng14{k25=0} for conv (f32[20,32,222,222]{3,2,1,0}, u8[0]{0}) custom-call(f32[20,3,224,224]{3,2,1,0}, f32[32,3,3,3]{3,2,1,0}, f32[32]{0}), window={size=3x3}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBiasActivationForward", backend_config={"cudnn_conv_backend_config":{"activation_mode":"kRelu","conv_result_scale":1,"leakyrelu_alpha":0,"side_input_scale":0},"force_earliest_schedule":false,"operation_queue_id":"0","wait_on_operation_queues":[]}
2025-03-19 18:55:24.019714: I external/local_xla/xla/service/gpu/autotuning/conv_algorithm_picker.cc:557] Omitted potentially buggy algorithm eng14{k25=0} for conv (f32[20,64,109,109]{3,2,1,0}, u8[0]{0}) custom-call(f32[20,32,111,111]{3,2,1,0}, f32[64,32,3,3]{3,2,1,0}, f32[64]{0}), window={size=3x3}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBiasActivationForward", backend

7/7 ━━━━━━━━━━━━━━━━━━━━ 19s 2s/step - accuracy: 0.4037 - loss: 3.2586 - val_accuracy: 0.5500 - val_loss: 0.8759
Epoch 2/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 6s 844ms/step - accuracy: 0.5160 - loss: 0.8378 - val_accuracy: 0.5500 - val_loss: 0.6740
Epoch 3/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 6s 796ms/step - accuracy: 0.5749 - loss: 0.6743 - val_accuracy: 0.3500 - val_loss: 0.6797
Epoch 4/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 6s 786ms/step - accuracy: 0.6780 - loss: 0.6231 - val_accuracy: 0.6000 - val_loss: 0.6369
Epoch 5/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 6s 834ms/step - accuracy: 0.5836 - loss: 0.6742 - val_accuracy: 0.6000 - val_loss: 0.6368
Epoch 6/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 6s 910ms/step - accuracy: 0.5808 - loss: 0.6255 - val_accuracy: 0.5500 - val_loss: 0.6458
Epoch 7/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 5s 881ms/step - accuracy: 0.6563 - loss: 0.6075 - val_accuracy: 0.5000 - val_loss: 0.7213
Epoch 8/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 6s 784ms/step - accuracy: 0.6624 - loss: 0.6146 - val_accuracy: 0.5500 - val_loss: 0.6916
Epoch

In [8]:
# Save the model
model.save('check.h5')


In [9]:
print(model.input_shape)


(None, 224, 224, 3)


In [ ]:
import numpy as np
import os
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array
from PIL import Image

# Load the trained model
model_path = 'check.h5'  # Ensure the model file exists
if not os.path.exists(model_path):
    print(f"❌ Error: Model file '{model_path}' not found.")
    exit(1)

try:
    model = load_model(model_path)
    print("✅ Model loaded successfully.")
except Exception as e:
    print(f"❌ Error loading model: {e}")
    exit(1)

# Function to preprocess and predict currency authenticity
def predict_currency(image_path):
    """ Reads an image file, preprocesses it, and predicts the currency class. """
    try:
        if not os.path.exists(image_path):
            return f"❌ Error: File '{image_path}' does not exist."

        # Load and preprocess the image
        img = Image.open(image_path).convert('RGB')
        img = img.resize((224, 224))
        img = img_to_array(img) / 255.0
        img = np.expand_dims(img, axis=0)

        # Make prediction
        predictions = model.predict(img)
        predicted_class = np.argmax(predictions)  # Get the class index
        confidence = np.max(predictions) * 100  # Get the highest probability

        # Class Labels (Make sure these match your dataset)
        class_labels = {0: "Fake", 1: "Real", 2: "Damaged", 3: "Unknown"}

        return f"🔹 Prediction: {class_labels[predicted_class]} ({confidence:.2f}% confidence)"

    except Exception as e:
        return f"❌ Error processing image: {e}"



# Ask for an image file path
image_path = input("Enter the image file path: ").strip()

if image_path:
    result = predict_currency(image_path)
    print(result)
else:
    print("❌ No file path provided.")


In [20]:
tf.config.experimental.reset_memory_stats('GPU:0')
